# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import glob
import scipy.stats
import scipy.signal
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

# Define global frequency domain parameters
Fs = 125            # the sampling frequency
fmin = 40/60        # [Hz]
fmax = 240/60       # [Hz] 

model = None

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]



def LoadRefDataFile(ref_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadRefDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
     # load the reference signal
   
    data = sp.io.loadmat(ref_fl)['BPM0']
    return data.reshape(-1)

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """

    global model
    if model is None:
        # Train the model
        model = Train()
    
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def LoadTrainingDataset():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns: targets and features
    
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    targets, features = [], []
    for data_fl, ref_fl in (zip(data_fls, ref_fls)):
        
        # Load data using LoadTroikaDataFile
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
       
        # Load Ground Truth
        bpm0 = LoadRefDataFile(ref_fl)

        start, end = GetSigIntervals(max(len(ppg),len(bpm0)),Fs=125)
    
        for i, s in enumerate(start):
            e = end[i]
        
            # Bandpass filtering the signals
            ppg_bpf  = BPF(ppg[s:e])
            accx_bpf = BPF(accx[s:e])
            accy_bpf = BPF(accy[s:e])
            accz_bpf = BPF(accz[s:e])

            # Get features
            feature = Featurize(ppg_bpf,accx_bpf,accy_bpf,accz_bpf)

            targets.append(bpm0[i])
            features.append(feature)
            
    targets = np.array(targets)
    features = np.array(features)
    return targets, features


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """ Calculates mean absolute errors and confidences
    
    Args:
        data_fl: (string) Path to the signal file
        ref_fl: (string) Path to the reference signal file
        
    Returns:
        (np.array) Mean absolute errors
        (np.array) Confidences
        
    Algorithm steps:
      * The signal is split into equal non-overlapping time-intervals, synchronising the sensor signals and the reference signal.
      * Perform band-pass filtering on the signals to limit the pulse rate range to 40-240 bpm.
      * Extract the frequency domain features: highest-peak frequencies and the energies of those frequencies for each of the four sensor signals (PPG and 3-axes accelerometers).
      * A model is built and trained based on Random Forest Regression that takes the frequency features and predicts the pulse rate.
      * Compute the confidence in the prediction by measuring the approximate pulse energy (integration around the spectral peak).
      * The mean absolute error is used as a metric to evaluate the performance of the model.
      * The algorithm produces a pulse rate estimate and a confidence value at least every 2 seconds.
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Load Ground Truth
    bpm0 = LoadRefDataFile(ref_fl)
    #print('Ground truth dimensions: ' + str(len(bpm0)))
    #REF_01_TYPE01
    
    out1 = []
    out2 = []
    
    # If model has not been trained, we will just collect data: (out1,out2)=(targets,features)
    # If model has been trained, use it to predit bpm and give: (out1,out2)=(mae,confidence)
  
    #start_indices, end_indices = get_start_end(len(accx), len(bpm0))
    start, end = GetSigIntervals(max(len(ppg),len(bpm0)),Fs)
    
    for i, s in enumerate(start):
        e = end[i]
        
        # Bandpass filtering the signals
        ppg_bpf  =  BPF(ppg[s:e])
        accx_bpf = BPF(accx[s:e])
        accy_bpf = BPF(accy[s:e])
        accz_bpf = BPF(accz[s:e])

        # Get features
        feature = Featurize(ppg_bpf,accx_bpf,accy_bpf,accz_bpf)

        # Get prediction
        bpm = model.predict(np.reshape(feature, (1, -1)))[0]
        
        # Compute the FFT
        freq, fft_mag = ComputeFourierTransform(ppg_bpf,Fs)

        # Compute the confidence
        freq_est = bpm/60
        conf = GetConfidenceMax(freq,fft_mag,freq_est)

        out1.append(np.abs(bpm - bpm0[i]))
        out2.append(conf)
    
    out1 = np.array(out1)
    out2 = np.array(out2)
    # Return mean absolute error and confidence as a 2-tuple of numpy arrays.
    return out1, out2

def BPF(signal, fmin=40/60, fmax=240/60, fs=125):
    """Bandpass filter the signal between 0.67 and 4 Hz."""
    b, a = sp.signal.butter(3, (fmin, fmax), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def GetConfidenceMax(freq,fft_mag,freq_est,n_samples=6):
    """
    Get the confidence value for a particular frequency.
    Integrates the power spectrum around that frequency and normalizes it to the total power
    """  
    tot_power = sum(fft_mag)
    p = (np.abs(freq - freq_est)).argmin()
    confidence = np.sum(fft_mag[p - n_samples: p + n_samples])/tot_power
    return confidence

def GetSigIntervals(N,Fs=125,window_length_s = 10,window_shift_s = 2):
    """
    Define the window intervals in time domain.
    """
    
    window_length = window_length_s * Fs
    window_shift = window_shift_s * Fs
    start = []
    end = []
    for i in range(0, N - window_length + window_shift, window_shift):
        #window = ppg[i: i + window_length]
        start.append(i)
        end.append(i+window_length)
    
    return (start, end)

def ComputeFourierTransform(sig,Fs):
    """
    Get the frequency components and signal spectrum, when sampling at Fs.
    """
    freq = np.fft.rfftfreq(len(sig),1/Fs)
    fft_mag = np.abs(np.fft.rfft(sig))
    return freq, fft_mag 


def Train():
    """
    Define and train a RandomForest model to predit the pulse rate from the frequency features of the measured signals.
    """
    model = RandomForestRegressor(n_estimators=100,max_depth=15)
    
    lf = KFold(n_splits=5)
    targets, features = LoadTrainingDataset()
    splits = lf.split(features,targets)
    
    for i, (train_idx, test_idx) in enumerate(splits):
        X_train, y_train = features[train_idx], targets[train_idx]
        X_test, y_test = features[test_idx], targets[test_idx]
        model.fit(X_train, y_train)
    
    return model
    

def Featurize(ppg, accx, accy, accz, Fs=125):
    """ Create features """
    
    verbose = False
    
    freq_p, mag_p, conf_p = GetPeakFreq(BPF(ppg), Fs,verbose)
    freq_x, mag_x, conf_x = GetPeakFreq(BPF(accx),Fs,verbose)
    freq_y, mag_y, conf_y = GetPeakFreq(BPF(accy),Fs,verbose)
    freq_z, mag_z, conf_z = GetPeakFreq(BPF(accz),Fs,verbose)

    return [freq_p[0], freq_x[0], freq_y[0], freq_z[0], \
            conf_p[0], conf_x[0], conf_y[0], conf_z[0] ]

def GetPeakFreq(sig,Fs,verbose=False):
    """ Get frequencies and their energies in order of their spectral magnitude """
    # Compute the FFT
    freq, fft_mag = ComputeFourierTransform(sig,Fs)

    # Compute frequency domain features
    
    # Single max-peak search
    # peaks  = np.max(fft_mag)
    # freq_peaks = freq[np.argmax(fft_mag)]
    
    # Get the peaks in descending order and their indices
    zipped = sorted(zip(fft_mag,freq), key = lambda x: x[0], reverse=True)
    
    # Using list comprehension to perform Unzipping
    unzipped_list = [[ i for i, j in zipped ],[ j for i, j in zipped ]]
    peaks = unzipped_list[0]
    freq_peaks = unzipped_list[1]
    
    # Search and reduce number of peaks we're going to look at (first 10 for instance)
    min_df = 4*(freq[1]-freq[0])
    peaks_selected = [peaks[0]]
    freq_peaks_selected = [freq_peaks[0]]
    n = 1
    while len(peaks_selected)<10:
        l = [abs(freq_peaks[n]-f) for f in freq_peaks_selected]
        if all([d > min_df  for d in l]):
            peaks_selected.append(peaks[n])
            freq_peaks_selected.append(freq_peaks[n])
        n = n+1
    
    peaks = peaks_selected
    freq_peaks = freq_peaks_selected
    
    if verbose:
        plt.plot(freq,fft_mag)
        plt.plot(freq_peaks,peaks,'ro')

        #plt.xlim([fmin,fmax])
        plt.xlim([0,4])
        plt.grid('on')
        plt.show()

    
    confidence = GetConfidence(freq,fft_mag,freq_peaks)
    
    return freq_peaks, peaks, confidence


def GetConfidence(freq,fft_mag,freq_peaks,n_samples=6):
    """ 
    For each of the frequencies in peaks measure the confidence as the 
    power within an integration bandwidth (a percentage of the total 
    bandwidth) around those peaks. Normalized by the total signal power.
    """
    #n_samples = int(percentage*len(fft_mag))
    #print('n_samples'+str(n_samples))
    
    # Pad the spectrum in case our peaks are at the edges of the spectrum
    #fft_mag  = [0]*n_samples + fft_mag + [0]*n_samples
    #freq     = [0]*n_samples + freq    + [0]*n_samples
    
    tot_power = sum(fft_mag)

    idx_peaks = [np.where(freq==x)[0][0] for x in freq_peaks]   

    #print('freq[idx_peaks]')
    #print(freq[idx_peaks])
    #print('freq_peaks')
    #print(freq_peaks)

    confidence = []
    for p in idx_peaks:
        confidence.append(np.sum(fft_mag[p - n_samples: p + n_samples])/tot_power)
    
    #p = idx_peaks[0]
    #plt.plot(freq,fft_mag)
    #plt.plot(freq[p - n_samples: p + n_samples],fft_mag[p - n_samples: p + n_samples],'r')
    #plt.xlim([0.6,4])
    #plt.title(freq[p])
    #plt.show()
    
    return confidence

    
Evaluate()

4.904736979251779